# Этап 1: Сбор метаданных

**Файл:** `metadata_parse.ipynb`

Собираем метаданные всех статей через arXiv API

## Особенности реализации

- **Помесячный сбор** — обход лимитов
- **Фильтрация по primary_category** — исключаем cross-listed статьи
- **Rate limiting** — задержка 3 секунды между запросами
- **Retry logic** — 3 попытки при ошибках

---

In [1]:
import arxiv
import pandas as pd
import calendar
import time
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

In [2]:
# ===== КОНФИГУРАЦИЯ =====
CAT = 'cs.CL'
YEAR = 2025
OUTPUT_FILE = f"arxiv_NLP_{YEAR}_metadata.csv"

client = arxiv.Client(
    page_size=1000,
    delay_seconds=3,
    num_retries=5
)

In [3]:
all_papers = []

print(f"Парсим статьи за {YEAR} год по категории {CAT}")

for month in range(1, 13):
    last_day = calendar.monthrange(YEAR, month)[1] # Получаем последний день месяца
    
    start_str = f"{YEAR}{month:02d}01000000"
    end_str = f"{YEAR}{month:02d}{last_day}235959"
    
    query = f"cat:{CAT} AND submittedDate:[{start_str} TO {end_str}]"
    search = arxiv.Search(
        query=query,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    month_name = calendar.month_name[month]
    print(f'Парсим: {month_name}...', end=' ')
    
    counter = 0
    for paper in client.results(search):
        if paper.primary_category == CAT:
            all_papers.append({
                'arxiv_id': paper.get_short_id(),
                'title': paper.title,
                'authors': ', '.join([a.name for a in paper.authors]),
                'summary': paper.summary.replace('\n', ' '),
                'primary_category': paper.primary_category,
                'published': paper.published.strftime('%Y-%m-%d'),
                'updated': paper.updated.strftime('%Y-%m-%d'),
                'entry_id': paper.entry_id
            })
            counter += 1
    print(f'найдено {counter} статей.')

    time.sleep(1)
print(f'Парсинг завершён.')

Парсим статьи за 2025 год по категории cs.CL
Парсим: January... найдено 924 статей.
Парсим: February... найдено 1832 статей.
Парсим: March... найдено 1354 статей.
Парсим: April... найдено 1168 статей.
Парсим: May... найдено 2134 статей.
Парсим: June... найдено 1670 статей.
Парсим: July... найдено 1185 статей.
Парсим: August... найдено 1326 статей.
Парсим: September... найдено 1587 статей.
Парсим: October... найдено 1857 статей.
Парсим: November... найдено 1128 статей.
Парсим: December... найдено 907 статей.
Парсинг завершён.


In [4]:
df = pd.DataFrame(all_papers)
df['html_url'] = 'https://arxiv.org/html/' + df['arxiv_id'].astype(str)

Path('../data/metadata').mkdir(parents=True, exist_ok=True)

df.to_csv(f"../data/metadata/{OUTPUT_FILE}", index=False, encoding='utf-8')
print(f"\nDONE. Сохранено {len(df)} статей в {OUTPUT_FILE}")


DONE. Сохранено 17072 статей в arxiv_NLP_2025_metadata.csv
